In [3]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

ModuleNotFoundError: No module named 'piplite'

In [2]:
%pip install numpy pandas seaborn



[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

In [6]:
!pip install folium

     -------------------------------------- 113.4/113.4 kB 3.3 MB/s eta 0:00:00
     ---------------------------------------- 93.9/93.9 kB 5.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import folium
import pandas as pd

In [8]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

In [10]:
import pandas as pd
import math

In [9]:
spacex_df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"
)

spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [11]:
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = launch_sites_df.groupby('Launch Site', as_index=False).first()
launch_sites_df

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS SLC-40,28.563197,-80.576820,1
2,KSC LC-39A,28.573255,-80.646895,1
3,VAFB SLC-4E,34.632834,-120.610745,0


In [12]:
nasa_coordinates = [29.559684888503615, -95.0830971930759]

site_map = folium.Map(location=nasa_coordinates, zoom_start=5)
site_map


In [13]:
folium.Circle(
    nasa_coordinates,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)

folium.Marker(
    nasa_coordinates,
    icon=folium.DivIcon(
        html='<div style="font-size: 12px; color:#d35400;"><b>NASA JSC</b></div>'
    )
).add_to(site_map)

site_map


In [14]:
for index, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    
    folium.Circle(
        coordinate,
        radius=1000,
        color='#0000ff',
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)
    
    folium.Marker(
        coordinate,
        icon=folium.DivIcon(
            html=f'<div style="font-size:12px; color:#0000ff;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

site_map


In [15]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map


In [16]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

spacex_df['marker_color'] = spacex_df['class'].apply(
    lambda x: 'green' if x == 1 else 'red'
)


In [17]:
for index, row in spacex_df.iterrows():
    folium.Marker(
        [row['Lat'], row['Long']],
        icon=folium.Icon(
            color='white',
            icon_color=row['marker_color'],
            icon='info-sign'
        )
    ).add_to(marker_cluster)

site_map
